In [13]:
from eval7 import Deck, Card, handtype, evaluate
from collections import Counter
import time

# from royalty_calculator import score_back, score_mid, score_front
from hand import Hand
# from simulation import solve_4_cards, solve_2_cards #, get_possible_slots, place_cards, is_drawing_dead

from itertools import product, permutations, combinations
from random import sample

In [ ]:
def is_drawing_dead(hand):
    eval_front = evaluate(hand.front)
    eval_mid = evaluate(hand.mid)
    eval_back = evaluate(hand.back)

    if len(hand.back) == 5 and (eval_mid > eval_back or eval_front > eval_back):
        return 1

    if len(hand.mid) == 5 and eval_front > eval_mid:
        return 1

In [31]:
def get_possible_slots(hand):
    front_slots = 3 - len(hand.front)
    mid_slots = 5 - len(hand.mid)
    back_slots = 5 - len(hand.back)

    return (0,) * front_slots + (1,) * mid_slots + (2,) * back_slots


def place_cards(hand, cards_to_place, slots_):
    free_slots = set(permutations(slots_, 2))
    cards_stack = combinations(cards_to_place, 2)

    hands_arr = []

    for cards, slots in product(cards_stack, free_slots):
        temp_hand = hand.copy()
        temp_hand.add_card(cards[0], slots[0])
        temp_hand.add_card(cards[1], slots[1])
        temp_hand.last_update = (cards, slots)
        hands_arr.append(temp_hand)

    return hands_arr

def is_drawing_dead(hand):
    eval_front = evaluate(hand.front)
    eval_mid = evaluate(hand.mid)
    eval_back = evaluate(hand.back)

    if len(hand.back) == 5 and (eval_mid > eval_back or eval_front > eval_back):
        return 1

    if len(hand.mid) == 5 and eval_front > eval_mid:
        return 1


def solve_2_cards(hand, cards):
    max_score = -6

    slots = get_possible_slots(hand)
    for current_hand in place_cards(hand, cards, slots):
        current_hand.evaluate_hand()
        score = current_hand.fantasy_score()
        if current_hand.is_fantasy:
            score += 8

        if score > max_score:
            max_score = score

    return max_score

def solve_4_cards(hand, cards, dead_cards):
    values_dic = {}

    used_cards = hand.front + hand.mid + hand.back + list(cards) + list(dead_cards)
    temp_deck = set(Deck().cards) - set(used_cards)


    slots = get_possible_slots(hand)
    for current_hand in place_cards(hand, cards, slots):
        if is_drawing_dead(current_hand):
            cards, slots = current_hand.last_update
            values_dic[(cards[0], cards[1], slots[0], slots[1])] = -6
        else:
            counter = 0
            cards, slots = current_hand.last_update
            values_dic.setdefault((cards[0], cards[1], slots[0], slots[1]), 0)

            for last_street_cards in combinations(temp_deck, 3):
                counter += 1
                score = solve_2_cards(current_hand, last_street_cards)
                values_dic[(cards[0], cards[1], slots[0], slots[1])] += score

            values_dic[(cards[0], cards[1], slots[0], slots[1])] /= counter

    return values_dic, counter

In [15]:
class Simulation:
    def __init__(self, hand, dead_cards):
        self._hand = hand.copy()
        self.dead_cards = dead_cards

    def simulate_8_cards(self, hand, cards, n_sim):
        used_cards = hand.front + hand.mid + hand.back + list(cards) + self.dead_cards
        temp_deck = set(Deck().cards) - set(used_cards)

        values_dic = {}

        for current_hand in place_cards(hand, cards):
            counter = 0
            cards, slots = current_hand.last_update
            values_dic.setdefault((cards[0], cards[1], slots[0], slots[1]), 0)

            all_combinations = list(combinations(temp_deck, 3))
            combos = sample(all_combinations, n_sim)
            for street_6_cards in combos:
                counter += 1
                res = self.simulate_6_cards(current_hand, street_6_cards, n_sim)

                values_dic[(cards[0], cards[1], slots[0], slots[1])] += max(res.values())

            values_dic[(cards[0], cards[1], slots[0], slots[1])] /= counter

        return values_dic
    
    
    def simulate_6_cards(self, hand, cards, n_sim):
        used_cards = hand.front + hand.mid + hand.back + list(cards) + self.dead_cards
        temp_deck = set(Deck().cards) - set(used_cards)

        values_dic = {}
        slots = get_possible_slots(hand)
        
        for current_hand in place_cards(hand, cards, slots):
            cards, slots = current_hand.last_update
            
            if is_drawing_dead(current_hand):
                values_dic[(cards[0], cards[1], slots[0], slots[1])] = -6
            else:
                counter = 0
                values_dic.setdefault((cards[0], cards[1], slots[0], slots[1]), 0)

                all_combinations = list(combinations(temp_deck, 3))
                combos = sample(all_combinations, n_sim)
                for street_4_cards in combos:
                    counter += 1
                    res = self.simulate_4_cards(current_hand, street_4_cards, n_sim)

                    values_dic[(cards[0], cards[1], slots[0], slots[1])] += max(res.values())

                values_dic[(cards[0], cards[1], slots[0], slots[1])] /= counter

        return values_dic

    
    
    def simulate_4_cards(self, hand, cards, n_sim):
        used_cards = hand.front + hand.mid + hand.back + list(cards) + self.dead_cards
        temp_deck = set(Deck().cards) - set(used_cards)

        values_dic = {}

        slots = get_possible_slots(hand)
        for current_hand in place_cards(hand, cards, slots):
            cards, slots = current_hand.last_update
            
            if is_drawing_dead(current_hand):
                values_dic[(cards[0], cards[1], slots[0], slots[1])] = -6
            else:
                values_dic.setdefault((cards[0], cards[1], slots[0], slots[1]), 0)
                counter = 0

                cards, slots = current_hand.last_update
                values_dic.setdefault((cards[0], cards[1], slots[0], slots[1]), 0)

                all_combinations = list(combinations(temp_deck, 3))
                combos = sample(all_combinations, n_sim)

                for last_street_cards in combos:
                    counter += 1
                    score = solve_2_cards(current_hand, last_street_cards)
                    values_dic[(cards[0], cards[1], slots[0], slots[1])] += score

                values_dic[(cards[0], cards[1], slots[0], slots[1])] /= counter

        return values_dic

In [16]:
combo = [Card('As'), Card('Js'), Card('Ts'), Card('6s'), Card('4s')]
evaluate(combo), handtype(evaluate(combo))

(84711490, 'Flush')

In [19]:
def makeFantasy(stack):
    maxHnd = {}
    scr = -1
    for i in combinations(stack, 3):
        live_cards = (cards for cards in stack if cards not in i)
        for j in combinations(live_cards, 5):
            live_cards = (cards for cards in stack if cards not in i+j)
            for k in combinations(live_cards, 5):
                h = {'low' : k, 'middle' : j, 'high' : i}
                dc = Score(h)
                if not dc.is_mucked() and dc.score_total() > scr:
                    maxHnd = h
                    scr = dc.score_total()
    bestHand = Score(maxHnd)
    
    print()
    print(bestHand)
    print('Total Score: ', scr)
    #print(count)
    #with open("Fantasy.txt", "a") as myfile:
        #myfile.write(str(scr)+'\n')

In [ ]:
%timeit score_mid(50632704)

In [ ]:
deck = Deck()
deck.shuffle()
combo = deck.deal(14)
startTime = time.time()
makeFantasy(combo)
print('time takes: ', str(time.time()-startTime))

In [ ]:
# Добавить проверку, что лучше - взять по максимуму очков сейчас или заложиться на фэнтэзи?

In [21]:
def get_average_value(filename):
    with open(filename) as f:
        content = [line.rstrip('\n') for line in f]
        
    sum_ = 0
    mucked = 0
    for row in content:
        points = int(row.split(', ')[-1])
        sum_ += points
        if points == -6:
            mucked += 1
        
    print('Количество игр:', len(content))
    print(f'Среднее количество очков: {sum_/len(content):.2f}')
    print(f'Процент busted игр: {mucked / len(content):.2f}')


In [22]:
get_average_value('log.txt')

Количество игр: 1157
Среднее количество очков: 2.06
Процент busted игр: 0.31


In [ ]:
# Среднее количество очков при раскладе Фэнтэзи:
# 10.55 за 8884 руки

# Количество игр: 1157
# Среднее количество очков: 2.06
# Процент busted игр: 0.31

In [ ]:
h = Hand([Card('Qh'), Card('Qc'), ], 
         [Card('5c'), Card('2h'), Card('5s'), ], 
         [Card('Jd'), Card('3d')])
cards = (Card('Th'), Card('Ts'), Card('5d'))

In [27]:
h = Hand([Card('Ad'), Card('Qd'), ], 
         [Card('6d'), Card('2h'), Card('2d'), Card('4s'),], 
         [Card('9d'), Card('7s'), Card('9c')])
cards = (Card('Qh'), Card('2c'), Card('8h'))

In [25]:
h = Hand([Card('Ad'), Card('Qs'), ], 
         [Card('6h'), Card('7c'), Card('6d')], 
         [Card('8d'), Card('8c'), Card('Jc'), Card('Jd'),])
cards = (Card('Th'), Card('6s'), Card('5s'))

In [23]:
h = Hand([Card('Qc'), Card('Kc'), Card('Kh')], 
         [Card('6d'), Card('6h'), Card('2s')], 
         [Card('Jd'), Card('Js'), Card('9d'),])
cards = (Card('6s'), Card('Kd'), Card('4d'))

In [40]:
solve_4_cards(h, cards, (Card('Ks'),))

({(Card("Qh"), Card("2c"), 0, 1): -2.049458365247839,
  (Card("Qh"), Card("2c"), 1, 2): -0.00656526972316446,
  (Card("Qh"), Card("2c"), 2, 1): -4.188751504540979,
  (Card("Qh"), Card("2c"), 2, 0): -0.007002954371375424,
  (Card("Qh"), Card("2c"), 0, 2): -3.1967392493708284,
  (Card("Qh"), Card("2c"), 2, 2): -0.02954371375424007,
  (Card("Qh"), Card("2c"), 1, 0): 0.24597877229456178,
  (Card("Qh"), Card("8h"), 0, 1): -6,
  (Card("Qh"), Card("8h"), 1, 2): -0.00656526972316446,
  (Card("Qh"), Card("8h"), 2, 1): -0.00656526972316446,
  (Card("Qh"), Card("8h"), 2, 0): -0.007002954371375424,
  (Card("Qh"), Card("8h"), 0, 2): -2.4246635299266877,
  (Card("Qh"), Card("8h"), 2, 2): -0.02954371375424007,
  (Card("Qh"), Card("8h"), 1, 0): 0.24597877229456178,
  (Card("2c"), Card("8h"), 0, 1): 0.24597877229456178,
  (Card("2c"), Card("8h"), 1, 2): -4.188751504540979,
  (Card("2c"), Card("8h"), 2, 1): -0.00656526972316446,
  (Card("2c"), Card("8h"), 2, 0): 0.002844950213371266,
  (Card("2c"), Card

In [41]:
sim = Simulation(h, [])
%time sim.simulate_4_cards(h, cards, 500)

CPU times: total: 219 ms
Wall time: 283 ms


{(Card("Qh"), Card("2c"), 0, 1): -2.486,
 (Card("Qh"), Card("2c"), 1, 2): -0.012,
 (Card("Qh"), Card("2c"), 2, 1): -4.106,
 (Card("Qh"), Card("2c"), 2, 0): 0.02,
 (Card("Qh"), Card("2c"), 0, 2): -3.036,
 (Card("Qh"), Card("2c"), 2, 2): -0.024,
 (Card("Qh"), Card("2c"), 1, 0): 0.204,
 (Card("Qh"), Card("8h"), 0, 1): -6,
 (Card("Qh"), Card("8h"), 1, 2): 0.0,
 (Card("Qh"), Card("8h"), 2, 1): 0.0,
 (Card("Qh"), Card("8h"), 2, 0): 0.016,
 (Card("Qh"), Card("8h"), 0, 2): -2.082,
 (Card("Qh"), Card("8h"), 2, 2): 0.0,
 (Card("Qh"), Card("8h"), 1, 0): 0.196,
 (Card("2c"), Card("8h"), 0, 1): 0.268,
 (Card("2c"), Card("8h"), 1, 2): -4.596,
 (Card("2c"), Card("8h"), 2, 1): 0.0,
 (Card("2c"), Card("8h"), 2, 0): 0.004,
 (Card("2c"), Card("8h"), 0, 2): 0.008,
 (Card("2c"), Card("8h"), 2, 2): -0.012,
 (Card("2c"), Card("8h"), 1, 0): -4.528}

In [ ]:
# Проверка руки:
# - Если в нижнем слоте 4 карты и не флэш/стрит-дро, тогда проверяем, чтобы карта, положенная в средний/верхний слот
#   не делала руку мёртвой

# - Для каждой комбинации карт вычисляем максимально и минимально возможное количество кушей. MiniMax value.
#   Из всез значений в узловой точке выбираем то, которое ведёт к максимальному результату. Учитываем вероятности.

# - Сделать раздельные проверки для 2-3-6 свободных слотов, чтобы не тратить лишнее время


In [ ]:
# Filtering
# - Вычисляем все возможные перестановки, но берём только лучшую часть из них, не надо считать все
# - Можно попробовать брать не только лучшие варианты, но и парочку случайных - на тему генетических алгоритмов
# - Проверяем, что карта, положенная в слот, или дополняет дрова, или усиливает руку до следующего ранга (evaluate)

In [ ]:
# Tree pruning
# - Если на каком-то этапе я понимаю, что текущий выбор уже лучше/хуже, чем предыдущий, можно дальше не считать, а брать
#   его по дефолту и сравнивать именно его с дальнейшими вариантами

In [ ]:
%time solved_dict = solve_4_cards(h, cards, dead_cards)

In [ ]:
%time sim_dict = sim.simulate_4_cards(h, cards, 200)

In [ ]:
print('Solution: {} - {}, {} - {}; Score: {:.2f}'.format(*max(solved_dict, key=solved_dict.get), max(solved_dict.values())))
print('Simulation: {} - {}, {} - {}; Score: {:.2f}'.format(*max(sim_dict, key=sim_dict.get), max(sim_dict.values())))

In [ ]:
# 3.75 s
# 100 ms

In [ ]:
def is_straight_possible(cards):
    score = evaluate(cards)
    no_pair_flag = handtype(score) == 'High Card'
    
    if not 1 < len(cards) < 5:
        raise AttributeError('Number of cards should be 1 < len(cards) < 5')
    
    if not no_pair_flag:
        return False
    
    values = list(card.rank for card in cards)
    values.sort()
    
    if max(values) - min(values) < 5:
        return True
    
    if values[-1] == 12 and values[-2] < 4:
        return True
    
    return False

In [ ]:
def is_flush_possible(cards):
    if not 1 < len(cards) < 5:
        raise AttributeError('Number of cards should be 1 < len(cards) < 5')

    return len(set(card.suit for card in cards)) == 1

In [ ]:
is_straight_possible(cards)

In [ ]:
is_flush_possible(cards)

In [ ]:
cards = (Card("2s"), Card("3s"))
values = list(card.rank for card in cards)
index_min = min(range(len(values)), key=values.__getitem__)
cards[index_min]